## Formulazione del problema

La natura intermittente e lo scarso controllo sulle condizioni del vento pongono lo stesso problema a tutti i gestori di rete nella loro integrazione per soddisfare la domanda corrente. Oltre a dover prevedere la domanda e bilanciarla con l'offerta, l'operatore di rete deve anche prevedere la disponibilità degli impianti di generazione eolica e solare nell'ora, nel giorno o nella settimana successivi. Oltre a frenare i benefici dell'energia rinnovabile, una programmazione errata degli impianti di generazione eolica può portare a prenotazioni non necessarie, a costi più elevati trasferiti ai consumatori e all'utilizzo di altre risorse energetiche più costose e inquinanti. Lavorare con dati reali è difficile a causa del rumore e dei periodi mancanti.


## Dettaglio del dataset

link dataset: https://www.kaggle.com/datasets/pravdomirdobrev/texas-wind-turbine-dataset-simulated

Le serie orarie annuali fornite sono state simulate utilizzando il software del National Renewable Energy Laboratory (NREL) per una località del Texas, negli Stati Uniti. Presenta una perfetta completezza dei dati e l'assenza di dati rumorosi; sfide che ostacolano le attività di previsione con i set di dati reali e distraggono dall'obiettivo. Il set di dati contiene diverse caratteristiche meteorologiche che possono essere analizzate e utilizzate come predittori.

## Dettagli del task
Prevedere la produzione di energia del giorno successivo = 24 passi avanti.

In particolare noi ci concentreremo solo sull'analisi decrittiva del dataset e sulla sua elaborazione con gli strumenti che abbiamo appreso. Faremo una prova di previsione con una baseline basata sul valore precedente.

### Import del Dataset

In [ ]:
import pandas as pd

df = pd.read_csv("Dati/TexasTurbine.csv")

### Esplorazione di base

In [ ]:
df.head()

In [ ]:
df.dtypes

In [ ]:
print(f"il dataset presenta {df.size} righe")

In [ ]:
df.describe()

In [ ]:
start = df["Time stamp"].iloc[0]
end = df["Time stamp"].iloc[-1]
print(f"Il periodo coperto va dal {start} al {end} di un anno ipotetico.")

## Data cleaning

In [ ]:
# count missing values
df.isna().sum()

In [ ]:
rename_dict = {'Time stamp':'ts',
'System power generated | (kW)': 'potenza',
'Wind speed | (m/s)': 'velocità',
'Wind direction | (deg)':'direzione',
'Pressure | (atm)': 'pressione',
"Air temperature | ('C)": 'temperatura'}

df = df.rename(columns=rename_dict)

In [ ]:
# Converte la colonna 'dt' in oggetti datetime
df['ts'] = pd.to_datetime(df['ts'], format='%b %d, %I:%M %p')

# Crea un nuovo DataFrame con l'indice impostato sulla colonna 'dt'
df_ts = df.set_index('ts')

In [ ]:
df_ts.head()

### Gestione dell'indice temporale

Quando si lavora con dati di serie temporali, è importante che le informazioni relative al tempo siano in un formato facilmente leggibile dal computer. È qui che gli oggetti datetime si rivelano utili. Convertendo la colonna 'Time stamp' da un oggetto stringa a un oggetto datetime, è possibile eseguire varie operazioni sui dati, come filtrare, aggregare e tracciare in base a intervalli di tempo come giorni, settimane, mesi e così via.

Ad esempio, possiamo facilmente raggruppare i dati per mese e calcolare la potenza media generata per mese, il che può aiutarci a identificare tendenze e modelli nei dati. Inoltre, gli oggetti datetime forniscono vari attributi e metodi che consentono di estrarre informazioni specifiche dalla data e dall'ora, come il mese, il giorno, l'ora, il minuto, ecc. Questo può aiutarci a eseguire analisi più granulari e a ottenere approfondimenti sui dati.

In generale, la conversione della colonna 'Time stamp' in un oggetto datetime facilita il lavoro con i dati delle serie temporali e consente di eseguire analisi e visualizzazioni più sofisticate.

La libreria datetime è molto utile per lavorare con dati di serie temporali, poiché offre metodi per effettuare la conversione tra diversi formati di data e tempo, la creazione di intervalli di tempo e l'accesso a parti specifiche di una data o ora. Inoltre, è possibile eseguire operazioni matematiche come la somma e la sottrazione di date e orari, rendendo la libreria datetime uno strumento molto versatile per l'analisi dei dati.

Per trasformare correttamente la colonna 'Time stamp' in un oggetto datetime, possiamo utilizzare il metodo pd.to_datetime di Pandas, specificando il formato della stringa di input utilizzando i codici di formato. In questo caso, il formato corretto della stringa è '%b %d, %I:%M %p', dove:

'%b' rappresenta il mese abbreviato in tre lettere (es. Jan per Gennaio)
'%d' rappresenta il giorno del mese (01-31)
'%I' rappresenta l'ora in formato 12 ore (01-12)
'%M' rappresenta i minuti (00-59)
'%p' rappresenta l'indicatore AM o PM

In [ ]:
df_ts

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(50, 10))
plt.plot(df_ts['potenza'])
plt.title('potenza nel tempo')
plt.show()

In [ ]:
plt.figure(figsize=(30, 10))
cond = df_ts.index >'1900-12-01 00:00:00'
plt.plot(df_ts['potenza'].loc[cond], '-o')
plt.title('potenza nel tempo (dicembre)')
plt.show()

## Analisi aggregata
possiamo effettuare un'analisi aggregata sulla produzione totale della giornata e su altre variabili prese come media della giornata. Per fare ciò, possiamo usare il metodo groupby() di Pandas per raggruppare i dati per giorno e calcolare la somma della potenza generata e la media delle altre variabili per ciascun giorno.

In [ ]:
# Estraiamo la data come una nuova colonna
df_ts['giorno'] = df_ts.index.date

# Calcoliamo la somma della potenza generata e la media delle altre variabili per giorno
daily_data = df_ts.groupby('giorno').agg({'potenza': 'sum',
                                      'velocità': 'mean',
                                      'direzione': 'mean',
                                      'pressione': 'mean',
                                      'temperatura': 'mean'}).reset_index()
# Visualizziamo le prime righe del nuovo dataframe
print(daily_data.head())


In [ ]:
# Converte la colonna 'dt' in oggetti datetime
daily_data['giorno'] = pd.to_datetime(daily_data['giorno'])

# Crea un nuovo DataFrame con l'indice impostato sulla colonna 'dt'
daily_data = daily_data.set_index('giorno')

In [ ]:
plt.figure(figsize=(30, 10))
cond = daily_data.index >'1900-12-01 00:00:00'
plt.plot(daily_data['potenza'].loc[cond],'-o')
plt.title('potenza totale giornaliera (dicembre)')
plt.show()

In [ ]:
plt.figure(figsize=(30, 10))
plt.plot(daily_data['potenza'],'-o')
plt.title('potenza totale giornaliera')
plt.show()

In [ ]:
import matplotlib.pyplot as plt


df.hist(figsize=(10, 8))
plt.tight_layout()
plt.show()

In [ ]:
plt.scatter(df_ts['velocità'], df_ts['potenza'])
plt.xlabel('Velocità')
plt.ylabel('Potenza')
plt.title('Scatter plot: Velocità vs Potenza')
plt.show()


## Regressione 

Ipotizziamo di avere delle previsioni della velocità del vento e di voler prevedere che potenza avremo a disposizione la prossima settimana, giorno per giorno. 

In [ ]:
# Importare le librerie necessarie
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import matplotlib.dates as mdates

# Importare il dataset
df = pd.read_csv("Dati/TexasTurbine.csv")

# Preprocessing dei dati
rename_dict = {'Time stamp':'ts',
               'System power generated | (kW)': 'potenza',
               'Wind speed | (m/s)': 'velocità',
               'Wind direction | (deg)':'direzione',
               'Pressure | (atm)': 'pressione',
               "Air temperature | ('C)": 'temperatura'}

df = df.rename(columns=rename_dict)
df['ts'] = pd.to_datetime(df['ts'], format='%b %d, %I:%M %p')
df_ts = df.set_index('ts')

# Estrazione dei dati giornalieri
df_ts['giorno'] = df_ts.index.date
daily_data = df_ts.groupby('giorno').agg({'potenza': 'sum',
                                          'velocità': 'mean'}).reset_index()

# Creazione della variabile di input X e della variabile di output y
X = daily_data['velocità'].values.reshape(-1, 1)
y = daily_data['potenza'].values

# Split dei dati in training set e test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Creazione del modello di regressione lineare
linear_model = LinearRegression()
linear_model.fit(X_train, y_train)

# Creazione del modello di Random Forest
rf_model = RandomForestRegressor(random_state=42)
rf_model.fit(X_train, y_train)

# Calcolo delle previsioni dei modelli
y_pred_linear = linear_model.predict(X_test)
y_pred_rf = rf_model.predict(X_test)

# Calcolo degli errori quadratici medi (MSE)
mse_linear = mean_squared_error(y_test, y_pred_linear)
mse_rf = mean_squared_error(y_test, y_pred_rf)

# Stampa dei valori degli MSE
print("Errore quadratico medio (MSE) - Regressione lineare:", mse_linear)
print("Errore quadratico medio (MSE) - Random Forest:", mse_rf)

# Creazione della baseline usando la media della potenza generata dell'ultima settimana
last_week_data = daily_data.tail(7)
baseline_forecast = pd.DataFrame(index=last_week_data['giorno'])
baseline_forecast['potenza'] = last_week_data['potenza'].mean()

# Plot dei risultati delle previsioni per l'ultima settimana dei dati giornalieri
plt.figure(figsize=(12, 6))
plt.plot(last_week_data['giorno'], last_week_data['potenza'], color='blue', label='Dati effettivi')
plt.plot(last_week_data['giorno'], linear_model.predict(last_week_data['velocità'].values.reshape(-1, 1)), color='red', linestyle='--', label='Regressione lineare')
plt.plot(last_week_data['giorno'], rf_model.predict(last_week_data['velocità'].values.reshape(-1, 1)), color='green', linestyle='--', label='Random Forest')
plt.plot(baseline_forecast.index, baseline_forecast['potenza'], color='orange', linestyle='--', label='Baseline (Media)')
plt.xlabel('Data')
plt.ylabel('Potenza generata (kW)')
plt.title('Previsione della potenza generata')
plt.gca().xaxis.set_major_locator(mdates.DayLocator())
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%b %d'))
plt.legend()
plt.show()


In [ ]:
print(f"Linear model: \n potenza_generata = {round(linear_model.intercept_)} + \
      {round(linear_model.coef_[0])} * velocità_vento")

In [ ]:
# Crea una sequenza di valori di velocità del vento all'interno dell'intervallo coperto dai dati del vento reale.
wind_power_sequence = np.linspace(daily_data['velocità'].min(), daily_data['velocità'].max(), num=100)

# Genera valori di potenza stimati utilizzando i modelli
baseline_power = np.full_like(wind_power_sequence, last_week_data['potenza'].mean())
linear_regression_power = linear_model.predict(wind_power_sequence.reshape(-1, 1))
random_forest_power = rf_model.predict(wind_power_sequence.reshape(-1, 1))
plt.figure(figsize=(10, 6))

# Plot dati reali
plt.scatter(daily_data['velocità'], daily_data['potenza'], color='blue', label='Actual Data',s = 5)

# Plot dati stimati attraverso i modelli

plt.plot(wind_power_sequence, baseline_power, color='orange', linestyle='--', label='Baseline (Mean)')
plt.plot(wind_power_sequence, linear_regression_power, color='red', label='Linear Regression', linewidth=2)
plt.plot(wind_power_sequence, random_forest_power, color='green', label='Random Forest', linewidth=2)

plt.xlabel('Wind Power (m/s)')
plt.ylabel('Estimated Power (kW)')
plt.title('Estimated Power vs. Wind Power')
plt.legend()
plt.show()